In [1]:
import pandas as pd

dataset_id = "1KfvUPTtutsF63NxNXLtoFJQRASm1WwP3" #dataset-cleaned.csv
url = f"https://drive.google.com/uc?id={dataset_id}"

df = pd.read_csv(url)
#sentences = df['Sentence'].tolist()

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import LlamaTokenizer, AutoModelForCausalLM
import torch


model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [4]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [5]:
import re

def translate(sentence):
    prompt = (
        f'You are an expert at translating text from non modern Italian to modern Italian. Non modern Italian is a form of the Italian language that was directly derived from Ancient Latin. Provide the translation of the following sentence written between 1250 and 1350, specifically the Tuscan and Umbrian variety, into modern Italian.\n'
        f'Only return the translated sentence enclosed in double quotes. No notes, no explanations.\n\n'
        f'Old Italian: "{sentence}"\n'
        f'Modern Italian:'
    )

    output = pipe(prompt, max_new_tokens=100, do_sample=False)[0]["generated_text"]

    # Extract only what follows "Modern Italian:"
    response = output.split("Modern Italian:")[-1].strip()
    # Extract only what is inside the quotation marks ""
    match = re.search(r'"([^"]+)"', response)

    if match:
        translation = match.group(1).strip()
    else:
        # Fallback: use entire response, clean newlines
        translation = response.strip().split('\n')[0]

    return translation


In [6]:
df['Translation'] = df['Sentence'].apply(translate)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more de

In [7]:
df

,Author,Date,Region,Sentence,Translation
0,Brunetto Latini,1260-61,fior.,quella guerra ben fatta l' opera perché etc. E...,"Quella guerra fu fatta bene, l'opera, perché e..."
1,Bono Giamboni,1292,fior.,"crudele, e di tutte le colpe pigli vendetta, c...","Crudele e vendicativo, come la legge vuole, no..."
2,Valerio Massimo (red. V1,1336,fior.,Non d' altra forza d' animo fue ornato Ponzio ...,Non fu mai ornato da altra forza d'animo Ponzi...
3,Lucano volg. (ed. Marinoni),1330/40,prat.,Se questo piace a tutti e se 'l tempo hae biso...,Se questo piace a tutti e se il tempo ha bisog...
4,Brunetto Latini,1260-61,fior.,Officio di questa arte pare che sia dicere app...,L'ufficio di questa arte sembra essere dire ap...
...,...,...,...,...,...
92,Grazia di Meo,1343,tosc.,"Alchuno è riccho e gentile, ma lamentasi che v...","Alcuni sono ricchi e gentili, ma si lamentano ..."
93,Zanobi da Strada,1361,tosc.,"Pietro, essendogli mostrato in figura il popul...","Pietro, mostrando loro il popolo gentile, gli ..."
94,Declamazioni di Seneca,1392,tosc.,pregollo che lo liberasse di quella obbligazio...,"prego che lo liberi da quel debito, in cui egl..."
95,Giovanni da San Miniato,1415,tosc.,L'oro verrà dall'Aquilone. Che figuriamo noi p...,L'oro verrà dal Nord. Che figuriamo noi per il...


In [8]:
print(df["Translation"].iloc[73])

Chi non sa amare il prossimo come sé stesso già comincia a temere i giudizi di Dio.


In [9]:
# Salva in formato JSONL
df.to_json('giuminia-hw2_transl-llama-3-8B-Instruct-2.jsonl', orient='records', lines=True, force_ascii=False)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import shutil

source_path = '/content/giuminia-hw2_transl-llama-3-8B-Instruct-2.jsonl'
destination_path = '/content/drive/MyDrive/model_jsonl/giuminia-hw2_transl-llama-3-8B-Instruct-2.jsonl'


shutil.copy(source_path, destination_path)

'/content/drive/MyDrive/model_jsonl/giuminia-hw2_transl-llama-3-8B-Instruct-2.jsonl'